In [1]:
import pandas as pd
import numpy as np

import os
import json
import datetime

from matplotlib import pyplot as plt
from scipy import stats, special

import utils
import links
import generators
import estimation

from sklearn import mixture

%load_ext autoreload
%autoreload 2



# Definitions of non-intuitive things

#### What is X?
X is a matrix of weather conditions generated in 1-InputGeneration.ipynb

The data are synthetic, but they're based on statistical properties of real weather data.

#### What is a scenario?
Different "scenarios" represent different failure processes used to generate the data. These could be constant, piecewise linear, etc.

Sceanrios are pre-defined in "inputs/scenarios.csv". Right now they're completely made up. Next steps are to use realistic assumptions about the lifetime and failure properties of grid assets (ultimately this piece goes in 2-ScenarioGeneration.ipynb - but for now that notebook is a mess and accomplishes nothing)



In [2]:
fleet_size=10000
X = pd.read_csv('inputs/weather.csv', index_col='time')
X.head()



,Precip,Temp,Wind,DayPrecip,HotCalm,WindStorm
time,,,,,,
0,0.0,54.50,5.67,0.0,9.611993,0.0
1,0.0,53.46,3.85,0.0,13.885714,0.0
2,0.0,53.16,3.34,0.0,15.916168,0.0
3,0.0,52.57,3.62,0.0,14.522099,0.0
4,0.0,51.00,3.77,0.0,13.527851,0.0


In [31]:
scenarios = pd.read_csv('inputs/scenarios.csv', index_col='name')
        
for i, scenario in scenarios.iterrows():
    if i != 24:
        continue
    params = generators.params(scenario, fleet_size=fleet_size)
    failures, failure_rate = generators.failure_data(params, X, links.Link, fleet_size=fleet_size)
    if 'scenario%i'%(i) not in os.listdir('scenarios/'):
        os.mkdir(os.path.join('scenarios', 'scenario%i'%(i)))
        os.mkdir(os.path.join('scenarios', 'scenario%i'%(i), 'plots'))
        
    df = pd.DataFrame(index=failures.index)
    df['count'] = failures
    df['rate'] = failure_rate
    
    print df['count'].max()

    df.to_csv(os.path.join('scenarios','scenario%i'%(i),'failures.csv'), header=True)

    

71.0


# Scenario analysis
It takes quite a while to run through a scenario, let alone all of them. If you're just playing around, you can change "scenarios_list" to run only a subset of scenarios, or change "mcmc_stopping_criteria" to stop after fewer iterations. People will run ~1e5 or 1e6 iterations, but if the model is a good fit (big if, see 5.2) then it converges much faster.

Just a heads up. :-)

In [34]:

# USE RANDOM WALK
mcmc_random_walk = lambda theta, cov: pd.Series(stats.multivariate_normal.rvs(mean=theta, cov=cov),
                                           index=theta.index)

def hybrid_gibbs_sampler(theta, cov, design_lims=[40,50]):
    theta_star = pd.Series(index=theta.index)
    theta_star.loc['threshold.Wind'] = stats.uniform.rvs(*design_lims)
    for param in ['threshold.Precip','threshold.WindStorm']:
        if param in theta_star.index:
            key = param.split('.')[1]
            idx = X[key] > 0
            theta_star.loc[param] = stats.uniform.rvs(X[key][idx].min(), X[key][idx].max())

    for _var in var:
        A = 'slope.%s'%(_var)
        B = 'threshold.%s'%(_var)
        cov_inv = pd.DataFrame(np.linalg.inv(cov), columns=cov.keys(), index=cov.index)

        conditional_mean = theta.loc[A] + cov[A].loc[B]*cov_inv[B].loc[B]*(theta_star.loc[B] - theta.loc[B])
        conditional_var = cov[A].loc[A] - cov[A].loc[B]*cov_inv[B].loc[B]*cov[B].loc[A]

        theta_star.loc[A] = stats.norm.rvs(conditional_mean, np.sqrt(conditional_var))
    return theta_star

scenarios = pd.read_csv('inputs/scenarios.csv', index_col='name')

fleet_size = 10000
scenarios_list = scenarios.index
mcmc_stopping_criteria = lambda it: it<5e4

for i in scenarios_list:
    if i < 19:
        continue
    scenario = 'scenario%i'%(i)
    print scenario

    y = pd.read_csv(os.path.join('scenarios', scenario, 'failures.csv'), index_col='time')['count']
    
    print y.max()

    x_vars = [['Wind',],['Wind','Precip',],['Wind','Precip','WindStorm'],]
    for var in x_vars[1:]:
        model_name = '+'.join(var)
        
#         if os.path.exists(os.path.join('scenarios', scenario, 'chains', '%s.csv'%(model_name))):
#             continue
            
        _X = X[var]

        # -------------------------------
        # TUNING MCMC SAMPLER
        # Transition in the MCMC chain is a random walk where step size is gaussian and spherical
        # If the step size is too big, parameter updates are rarely accepted (poor mixing)
        # If the step size is too small, autocorrelation is high, convergence is slow
        # in both cases the chain needs to be longer.
        # Rule of thumb is to choose sigma so that acceptance rate is 25%
        
        
#         cov, p0 = estimation.transition_function(_X, y, links.Link(), fleet_size, tol=0.01, chain_size=1000)
#         estimation.save_chain_params(cov, p0, scenario, model_name)
        
        out = estimation.get_existing_transition_params(scenario, model_name)
#         out = []
        if len(out) == 0:
            cov, p0 = estimation.transition_function(_X, y, links.Link(), fleet_size, tol=0.01, chain_size=1000)
            estimation.save_chain_params(cov, p0, scenario, model_name)
        else:
            cov, p0 = out

#         p0, cov = links.Link().init_params(_X, y, fleet_size)
#         p0 = pd.io.json.json_normalize(p0)

        
        
        
                        

        params, acceptance = estimation.metropolis(p0, _X, y, links.Link(), fleet_size, 
                                                   jump=mcmc_random_walk, accept_reject=True, 
                                                   cov=cov, stop=mcmc_stopping_criteria)
        params['acceptance'] = acceptance

        if not os.path.exists(os.path.join('scenarios', scenario, 'chains')):
            os.mkdir(os.path.join('scenarios', scenario, 'chains'))

        params.to_csv(os.path.join('scenarios', scenario, 'chains', '%s.csv'%(model_name)))
        
        

scenario19
191.0
     slope.Precip  slope.Wind  threshold.Precip  threshold.Wind
0        9.603816    0.004659            0.0686              71
1        1.052009    0.013105            0.0686              71
2        0.496135    0.014990            0.0686              71
3        1.105126    0.012515            0.0686              71
4        0.588370    0.013959            0.0686              71
5        0.803289    0.013520            0.0686              71
6        0.000018    0.013718            0.0686              71
7        0.000019    0.014456            0.0686              71
8        0.491127    0.013085            0.0686              71
9        1.166816    0.013022            0.0686              71
10       1.304862    0.013973            0.0686              71
11       0.865780    0.012464            0.0686              71
12       0.931625    0.013366            0.0686              71
13       0.000019    0.014531            0.0686              71
14       0.359314    0.

estimation.py:54: RuntimeWarning: invalid value encountered in double_scalars
  


ValueError: probabilities contain NaN

In [30]:

# USE RANDOM WALK
mcmc_random_walk = lambda theta, cov: pd.Series(stats.multivariate_normal.rvs(mean=theta, cov=cov),
                                           index=theta.index)

def hybrid_gibbs_sampler(theta, cov, design_lims=[40,50]):
    theta_star = pd.Series(index=theta.index)
    theta_star.loc['threshold.Wind'] = stats.uniform.rvs(*design_lims)
    for param in ['threshold.Precip','threshold.WindStorm']:
        if param in theta_star.index:
            key = param.split('.')[1]
            idx = X[key] > 0
            theta_star.loc[param] = stats.uniform.rvs(X[key][idx].min(), X[key][idx].max())

    for _var in var:
        A = 'slope.%s'%(_var)
        B = 'threshold.%s'%(_var)
        cov_inv = pd.DataFrame(np.linalg.inv(cov), columns=cov.keys(), index=cov.index)

        conditional_mean = theta.loc[A] + cov[A].loc[B]*cov_inv[B].loc[B]*(theta_star.loc[B] - theta.loc[B])
        conditional_var = cov[A].loc[A] - cov[A].loc[B]*cov_inv[B].loc[B]*cov[B].loc[A]

        theta_star.loc[A] = stats.norm.rvs(conditional_mean, np.sqrt(conditional_var))
    return theta_star

scenarios = pd.read_csv('inputs/scenarios.csv', index_col='name')

fleet_size = 10000
scenarios_list = scenarios.index
mcmc_stopping_criteria = lambda it: it<1e5
extend=False

for i in scenarios_list:
    if i < 19:
        continue
    scenario = 'scenario%i'%(i)

    y = pd.read_csv(os.path.join('scenarios', scenario, 'failures.csv'), index_col='time')['count']
    
    if y.sum() == 0:
        continue

#     x_vars = [['Wind',],['Wind','Precip',],['Wind','Precip','WindStorm'],]
    x_vars = [['Wind','DayPrecip',],['Wind','DayPrecip','WindStorm'],]
    for var in x_vars:
        model_name = '+'.join(var)
        
#         if os.path.exists(os.path.join('scenarios', scenario, 'chains', '%s.csv'%(model_name))):
#             continue
            
        _X = X[var]

        # -------------------------------
        # TUNING MCMC SAMPLER
        # Transition in the MCMC chain is a random walk where step size is gaussian and spherical
        # If the step size is too big, parameter updates are rarely accepted (poor mixing)
        # If the step size is too small, autocorrelation is high, convergence is slow
        # in both cases the chain needs to be longer.
        # Rule of thumb is to choose sigma so that acceptance rate is 25%
        
        
#         cov, p0 = estimation.transition_function(_X, y, links.Link(), fleet_size, tol=0.01, chain_size=1000)
#         estimation.save_chain_params(cov, p0, scenario, model_name)
        
#         out = estimation.get_existing_transition_params(scenario, model_name)
        out = []
        if len(out) == 0:
            try:
                cov, p0 = estimation.transition_function(_X, y, links.Link(), fleet_size, tol=0.01, chain_size=1000)
            except ValueError:
                continue
            estimation.save_chain_params(cov, p0, scenario, model_name)
        else:
            cov, p0 = out
            if extend == True:
                p0 = pd.read_csv(os.path.join('scenarios', scenario, 'chains', '%s.csv'%(model_name)), 
                                 usecols=p0.index).iloc[-1]
            print p0
        params, acceptance = estimation.metropolis(p0, _X, y, links.Link(), fleet_size, 
                                                   jump=mcmc_random_walk, accept_reject=True, 
                                                   cov=cov, stop=mcmc_stopping_criteria)
        params['acceptance'] = acceptance

        if not os.path.exists(os.path.join('scenarios', scenario, 'chains')):
            os.mkdir(os.path.join('scenarios', scenario, 'chains'))

        if extend:
            df = pd.read_csv(os.path.join('scenarios', scenario, 'chains', '%s.csv'%(model_name)), index_col=0)
            df = df.append(params, ignore_index=True)
            df.to_csv(os.path.join('scenarios', scenario, 'chains', '%s.csv'%(model_name)))

        else:
            params.to_csv(os.path.join('scenarios', scenario, 'chains', '%s.csv'%(model_name)))
        
        
        

1000
1000
1000
1000
1000
1000
1000
1000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
1000
1000
1000
1000
1000
1000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000

In [64]:
cov, theta_star

NameError: name 'theta_star' is not defined

In [65]:
p0

slope.Precip       -10.070734
slope.Wind           0.340613
threshold.Precip     0.000000
threshold.Wind      46.502160
dtype: float64